In [10]:
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, AveragePooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.activations import relu, sigmoid, softmax

from tensorflow.keras import Input, Model

from tensorflow.keras.constraints import max_norm 
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K

from tensorflow.keras.datasets import cifar10

import sys
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib import colors
%matplotlib notebook

In [6]:
# set metaparameters
batch_size = 32
num_classes = 10
epochs = 25

# load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 23s 0us/step


In [7]:
# conversion and preprocessing
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_train /= 255
x_test /= 255

In [19]:
# build model
inputs = Input(shape=(32, 32, 3))
X = Conv2D(32, (3, 3), padding="same", input_shape=inputs.shape)(inputs)
X = relu(X)
X = Conv2D(32, (3, 3))(X)
X = relu(X)
X = MaxPooling2D(pool_size=(2, 2))(X)
X = Dropout(0.25)(X)

X = Conv2D(64, (3, 3), padding="same")(X)
X = relu(X)
X = Conv2D(64, (3, 3))(X)
X = relu(X)
X = MaxPooling2D(pool_size=(2, 2))(X)
X = Dropout(0.25)(X)

X = Flatten()(X)
X = Dense(512)(X)
X = relu(X)
X = Dropout(0.5)(X)
X = Dense(num_classes)(X)
outputs = softmax(X)

model = Model(inputs=inputs, outputs=outputs, name="cifar10")
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])
model.summary()

Model: "cifar10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
tf.nn.relu_19 (TFOpLambda)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
tf.nn.relu_20 (TFOpLambda)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 15, 15, 32)        0   

In [20]:
# train model
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), shuffle=True)

Epoch 1/25
1563/1563 [==============================] - 374s 239ms/step - loss: 2.3111 - accuracy: 0.1014 - val_loss: 2.3064 - val_accuracy: 0.1000
Epoch 2/25
 307/1563 [====>.........................] - ETA: 3:52 - loss: 2.3078 - accuracy: 0.1044

KeyboardInterrupt: 